# Artificial Neural Network

### Importing the libraries

In [71]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

## Part 1 - Data Preprocessing

### Importing the dataset

In [72]:
data = pd.read_excel('Folds5x2_pp.xlsx')
data['PE']

0       463.26
1       444.37
2       488.56
3       446.48
4       473.90
         ...  
9563    460.03
9564    469.62
9565    429.57
9566    435.74
9567    453.28
Name: PE, Length: 9568, dtype: float64

In [73]:
# Separate Target Variable and Predictor Variables
TargetVariable=['PE']
Predictors=['AT', 'V', 'AP','RH']

X=data[Predictors].values
y=data[TargetVariable].values

### Sandardization of data ###
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(6697, 4)
(6697, 1)
(2871, 4)
(2871, 1)


In [86]:

 
# create ANN model
model = Sequential()
 
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=10, input_dim=4, kernel_initializer='normal', activation='relu'))
 
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))
 
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
 
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')
 
# Fitting the ANN to the Training set
model.fit(X_train, y_train , batch_size = 20, epochs = 50, verbose=1)

Epoch 1/50
335/335 [==============================] - 1s 3ms/step - loss: 0.4062
Epoch 2/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0935
Epoch 3/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0729
Epoch 4/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0652
Epoch 5/50
335/335 [==============================] - 1s 3ms/step - loss: 0.0631
Epoch 6/50
335/335 [==============================] - 1s 3ms/step - loss: 0.0620
Epoch 7/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0615
Epoch 8/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0609
Epoch 9/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0607
Epoch 10/50
335/335 [==============================] - 1s 2ms/step - loss: 0.0606
Epoch 11/50
335/335 [==============================] - 1s 3ms/step - loss: 0.0603
Epoch 12/50
335/335 [==============================] - 1s 3ms/step - loss: 0.0602
Epoch 13/50
335/335 [====

In [87]:
Predictions=model.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)
 
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['PE']=y_test_orig
TestingData['PredictedPE']=Predictions
TestingData.head()

90/90 [==============================] - 0s 1ms/step


,AT,V,AP,RH,PE,PredictedPE
0,19.64,48.06,1014.81,74.96,455.27,456.403137
1,28.26,69.23,1013.01,42.10,436.31,438.101654
2,27.98,67.17,1007.32,75.29,440.68,434.301788
3,28.64,69.23,1013.11,37.13,434.40,438.217285
4,9.34,38.08,1019.56,67.74,482.06,481.322998


In [88]:
# Computing the absolute percent error
APE=100*(abs(TestingData['PE']-TestingData['PredictedPE'])/TestingData['PE'])
TestingData['APE']=APE
 
print('satisfaction factor:', 100-np.mean(APE))
TestingData.head()



satisfaction factor: 99.28152604559108


,AT,V,AP,RH,PE,PredictedPE,APE
0,19.64,48.06,1014.81,74.96,455.27,456.403137,0.248893
1,28.26,69.23,1013.01,42.10,436.31,438.101654,0.410638
2,27.98,67.17,1007.32,75.29,440.68,434.301788,1.447357
3,28.64,69.23,1013.11,37.13,434.40,438.217285,0.878749
4,9.34,38.08,1019.56,67.74,482.06,481.322998,0.152886


In [85]:
from sklearn.metrics import r2_score
r2_score(y_test_orig,Predictions)

0.9388861653265876